In [284]:
# Import Everything
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re
import json
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

In [293]:
# Connect to Firebase
cred = credentials.Certificate('./serviceAccount.json')
firebase_admin.initialize_app(cred)


db = firestore.client()

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [323]:
# Pick URL for cuisine and return image link, recipe link, instructions, ingredients, time to cook, servings, ratings
# Must use allrecipes.com world-cuisine type link

url = 'https://www.allrecipes.com/recipes/726/world-cuisine/european/spanish/'
#url = 'https://www.allrecipes.com/recipes/231/world-cuisine/european/'
#url = 'https://www.allrecipes.com/recipes/227/world-cuisine/asian/'
#url = 'https://www.allrecipes.com/recipes/723/world-cuisine/european/italian/'
response = requests.get(url)
#print(response)

doc_ref = db.collection(u'spanishcuisines')

In [324]:
soup = BeautifulSoup(response.text, "html.parser")
links = soup.select("a[data-click-id*=cardslot]")

In [169]:
#print(links)

[<a data-click-id="cardslot 1" data-content-provider-id="" data-internal-referrer-link="staff pick" href="https://www.allrecipes.com/recipe/259466/easy-caponata/">
<img alt="Easy Caponata Recipe - Caponata is a traditional vegetarian dish from Sicily with eggplant, zucchini, tomato, peppers, pine nuts, and raisins cooked in olive oil." class="fixed-recipe-card__img" data-lazy-load="" data-original-src="https://images.media-allrecipes.com/userphotos/300x300/4548530.jpg" src="https://images.media-allrecipes.com/ar/spacer.gif" style="display: inline;" title="Easy Caponata Recipe"/>
</a>, <a data-click-id="cardslot 2" data-content-provider-id="" data-internal-referrer-link="staff pick" href="https://www.allrecipes.com/recipe/255143/chef-johns-lemon-ice/">
<img alt="Chef John's Lemon Ice Recipe and Video - Using the classic Italian granita technique makes this an easy and refreshing summer treat. There's a delicious contrast between that light texture and bright, vibrant flavor." class="fix

In [325]:
# Get the recipe links off of the cuisine type

cardslotRegex = r"<a data-click-id=\"cardslot (\d+)"
imagesrcRegex = r'(?<=data-original-src=")([^"]+)(?=")'
tempmatch = ""
recipeLinks = []
recipeImageLinks =  []
for link in links:
    match = re.search(cardslotRegex,str(link)).group(0)
    if match == tempmatch:
        pass
    else:
        tempmatch = match
        recipeLink = link["href"]
        recipeLinks.append(recipeLink)
        recipeImageLink = re.search(imagesrcRegex,str(link)).group(0)
        recipeImageLinks.append(recipeImageLink)

In [326]:
###GET ALL RECIPES AND INFO###
data = {}
for (index, recipeLink) in enumerate(recipeLinks):
    data['image']=recipeImageLinks[index] 
    print(recipeLink)
    response = requests.get(recipeLink)
    #print(response)
    soup = BeautifulSoup(response.text, "html.parser")
    
    #Get Title
    try:
        title = soup.select("h1[class*=heading-content]")[0].text.strip()
    except:
        title = soup.select("h1")[0].text.strip()
    print(title)
    data['title']=title
    
    #Get Rating
    try:
        rating = soup.select("div[class*=rating-stars]")
        ratingRegex = r'(?<=data-ratingstars=")([^"]+)(?=")'
        rate = re.search(ratingRegex,str(rating)).group(0)
        print(rate)
    except:
        rate = soup.select("span[class*=review-star-text]")[0].text.strip()
        print(rate)
    data['rate']=rate
    
    #Get Time
    try:
        time = soup.select("span[class*=ready-in-time]")[0].text.strip()
        print(time)
    except:
        time = soup.select("div[class*=recipe-meta-item-body]")[2].text.strip()
        print(time)
    data['time']=time
    
    #Get Servings
    try:
        serving = soup.select("meta[itemprop*=recipeYield]")
        servingRegex = r'(?<=content=")([^"]+)(?=")'
        servingAmt = re.search(servingRegex,str(serving)).group(0)
        print(servingAmt)
    except:
        servingAmt = soup.select("div[class*=recipe-meta-item-body]")[5].text.strip()
        print(servingAmt)        
    data['serving']=servingAmt
    
    #Get Ingredients
    ingredients = soup.select("label[ng-class*=true:]")
    ingredientList = []
    for item in ingredients:
        ingredientRegex = r'(?<=title=")([^"]+)(?=")'
        try:
            ingredient = re.search(ingredientRegex,str(item)).group(0)
            ingredientList.append(ingredient)
        except:
            pass
    ingredients = soup.select("span[class*=ingredients-item-name]")
    for item in ingredients:
        ingredient = item.text.strip()
        ingredientList.append(ingredient)
    print(ingredientList)
    data['ingredients']=ingredientList
    
    #Get Directions
    directions = soup.select("span[class*=recipe-directions__list--item]")
    directionsList = []
    for item in directions:
        direction = item.text.strip()
        directionsList.append(direction)
    directions = soup.select("div[class*=section-body]")
    try:
        for item in directions:
            directionRegex = r'(?<=<p>)([^"]+)(?=</p>)'
            direction = re.search(directionRegex,str(item)).group(0)
            directionsList.append(direction)
    except:
        pass
    data['instructions']=directionsList
    print(directionsList)
    doc_ref.add(data)
    data={}
print("done")

https://www.allrecipes.com/recipe/230569/chef-johns-patatas-bravas/
Chef John's Patatas Bravas
4.60784292221069
1 h 30 m
4
['Dipping Sauce:', '1 clove garlic, minced, or more to taste', '1/2 teaspoon smoked paprika', '1 pinch salt', '1 cup mayonnaise', '1 tablespoon sherry vinegar, or more to taste', '1 teaspoon tomato paste', '1/4 teaspoon ground chipotle peppers', '1 pinch cayenne pepper, or more to taste', 'Spice Blend:', '1 tablespoon salt', '1/2 teaspoon ground black pepper', '1/2 teaspoon paprika', '1/2 teaspoon ground chipotle peppers', '2 quarts cold water', '1 tablespoon salt', '1 teaspoon smoked paprika', '1 teaspoon ground cumin', '2 bay leaves', '2 pounds russet potatoes, peeled and cut into 1-inch cubes', '2 cups vegetable oil for frying']
['Combine garlic, 1/2 teaspoon smoked paprika, and a pinch of salt in a mortar and pestle or food processor; process until smooth. Mix in mayonnaise, sherry vinegar, tomato paste, 1/4 teaspoon chipotle chile powder, and cayenne pepper un

Sangria! Sangria!
4.81994152069092
30 m
6
['1/2 cup brandy', '1/4 cup lemon juice', '1/3 cup frozen lemonade concentrate', '1/3 cup orange juice', '1 (750 milliliter) bottle dry red wine', '1/2 cup triple sec', '1 lemon, sliced into rounds', '1 orange, sliced into rounds', '1 lime, sliced into rounds', '1/4 cup white sugar (optional)', '8 maraschino cherries', '2 cups carbonated water (optional)']
['In a large pitcher or bowl, mix together the brandy, lemon juice, lemonade concentrate, orange juice, red wine, triple sec, and sugar. Float slices of lemon, orange and lime, and maraschino cherries in the mixture. Refrigerate overnight for best flavor. For a fizzy sangria, add club soda just before serving.\n                                    Watch Now', '']
https://www.allrecipes.com/recipe/20979/spanish-flan/
Spanish Flan
4.40610122680664
1 h 20 m
8
['1 cup white sugar', '3 eggs', '1 (14 ounce) can sweetened condensed milk', '1 (12 fluid ounce) can evaporated milk', '1 tablespoon vanill

IndexError: list index out of range

In [272]:
#### GET SINGLE RECIPE, only used for testing###
#Get Recipe Title
print(recipeLinks[7])
response = requests.get(recipeLinks[7])
print(response)
soup = BeautifulSoup(response.text, "html.parser")
try:
    title = soup.select("h1[class*=heading-content]")[1].text.strip()
except:
    title = soup.select("h1")[0].text.strip()
    print(title)

#Get Rating
try:
    rating = soup.select("div[class*=rating-stars]")
    ratingRegex = r'(?<=data-ratingstars=")([^"]+)(?=")'
    rate = re.search(ratingRegex,str(rating)).group(0)
    print(rate)
except:
    rate = soup.select("span[class*=review-star-text]")[0].text.strip()
    print(rate)

#Get Time
try:
    time = soup.select("span[class*=ready-in-time]")[0].text.strip()
    print(time)
except:
    time = soup.select("div[class*=recipe-meta-item-body]")[2].text.strip()
    print(time)

#Get Servings
try:
    serving = soup.select("meta[itemprop*=recipeYield]")
    servingRegex = r'(?<=content=")([^"]+)(?=")'
    servingAmt = re.search(servingRegex,str(serving)).group(0)
    print(servingAmt)
except:
    servingAmt = soup.select("div[class*=recipe-meta-item-body]")[5].text.strip()
    print(servingAmt)
    
#Get Ingredients 
ingredients = soup.select("label[ng-class*=true:]")
for item in ingredients:
    ingredientRegex = r'(?<=title=")([^"]+)(?=")'
    try:
        ingredient = re.search(ingredientRegex,str(item)).group(0)
        print(ingredient)
    except:
        pass
ingredients = soup.select("span[class*=ingredients-item-name]")
for item in ingredients:
    ingredient = item.text.strip()
    print(ingredient)

#Get Directions
directions = soup.select("span[class*=recipe-directions__list--item]")
directionsList = []
for item in directions:
    direction = item.text.strip()
    directionsList.append(direction)
directions = soup.select("div[class*=section-body]")
try:
    for item in directions:
        directionRegex = r'(?<=<p>)([^"]+)(?=</p>)'
        direction = re.search(directionRegex,str(item)).group(0)
        directionsList.append(direction)
except:
    pass
print(directionsList)

https://www.allrecipes.com/recipe/8887/chicken-marsala/
<Response [200]>
Chicken Marsala
Rating: 4.54 stars
30 mins
4 servings
¼ cup all-purpose flour for coating
½ teaspoon salt
¼ teaspoon ground black pepper
½ teaspoon dried oregano
4  skinless, boneless chicken breast halves - pounded 1/4 inch thick
4 tablespoons butter
4 tablespoons olive oil
1 cup sliced mushrooms
½ cup Marsala wine
¼ cup cooking sherry
['In a shallow dish or bowl, mix together the flour, salt, pepper and oregano. Coat chicken pieces in flour mixture.', 'In a large skillet, melt butter in oil over medium heat. Place chicken in the pan, and lightly brown.  Turn over chicken pieces, and add mushrooms. Pour in wine and sherry.  Cover skillet; simmer chicken 10 minutes, turning once, until no longer pink and juices run clear.']
